In [0]:
spark.conf.set(
    "fs.azure.account.key.projectadf1.dfs.core.windows.net",
    "CSiXNWhibZG0fqyBMd04Qn4++bAtOoS+HwZt5IWin/Pk9UrUimppMcGnXCzGUICGV/PCey4nuGzs+AStQ5MAfQ==")

## Read Source Tables

In [0]:
# Read appointments data
appointments_df = spark.read.format("parquet").load(f"abfss://source@projectadf1.dfs.core.windows.net/appointments")
patients_df =  spark.read.format("parquet").load(f"abfss://source@projectadf1.dfs.core.windows.net/patients")
doctors_df =  spark.read.format("parquet").load(f"abfss://source@projectadf1.dfs.core.windows.net/doctors")
billing_df = spark.read.format("parquet").load(f"abfss://source@projectadf1.dfs.core.windows.net/billing") 
procedures_df = spark.read.format("parquet").load(f"abfss://source@projectadf1.dfs.core.windows.net/medical_procedures")

## Build Base Data (Joins)

In [0]:
billing_df.printSchema()
billing_df.show(20, truncate=False)
billing_df.select("patient_id").distinct().show()


root
 |-- invoice_id: integer (nullable = true)
 |-- patient_id: integer (nullable = true)
 |-- items: string (nullable = true)
 |-- amount: double (nullable = true)

+----------+----------+-----------------------------+--------+
|invoice_id|patient_id|items                        |amount  |
+----------+----------+-----------------------------+--------+
|NULL      |894       |Immunizations                |956065.0|
|NULL      |448       |Cataract surgery             |188997.0|
|NULL      |618       |Pediatric surgery            |524091.0|
|NULL      |117       |Rhinoplasty                  |901393.0|
|NULL      |374       |Antibiotic therapy           |448957.0|
|NULL      |852       |Emergency surgical procedures|869041.0|
|NULL      |834       |Pediatric surgery            |492022.0|
|NULL      |140       |Hemodynamic monitoring       |644410.0|
|NULL      |459       |Emergency surgical procedures|295907.0|
|NULL      |734       |Immunotherapy (allergy shots)|766908.0|
|NULL      |26

In [0]:
from pyspark.sql.functions import col

# Join appointments with patients
appointment_patient_df = appointments_df.join(
    patients_df, 
    appointments_df.patient_id == patients_df.patient_id, 
    "inner"
).select(
    appointments_df["*"],
    patients_df["full_name"].alias("patient_name"),
    patients_df["email"].alias("patient_email")
)

# Join with doctors
appointment_doctor_df = appointment_patient_df.join(
    doctors_df,
    appointment_patient_df.doctor_id == doctors_df.doctor_id,
    "inner"
).select(
    appointment_patient_df["*"],
    doctors_df["doctor_name"],
    doctors_df["specialization"],
    doctors_df["contact_info"].alias("doctor_contact")
)

# Join with procedures
appointment_procedure_df = appointment_doctor_df.join(
    procedures_df,
    appointment_doctor_df.appointment_id == procedures_df.appointment_id,
    "left"
).select(
    appointment_doctor_df["*"],
    procedures_df["procedure_name"],
    procedures_df["procedure_id"]
)

# Join with billing (aggregate billing by appointment)
from pyspark.sql.functions import sum, coalesce, lit

billing_agg_df = billing_df.groupBy("patient_id").agg(
    sum("amount").alias("total_amount"),
    sum(coalesce(col("amount"), lit(0))).alias("total_billing")
)

appointment_summary_base_df = appointment_procedure_df.join(
    billing_agg_df,
    appointment_procedure_df.patient_id == billing_agg_df.patient_id,
    "left"
).drop(billing_agg_df.patient_id)


## Add Day Name (Text Format of Date)

In [0]:
from pyspark.sql.functions import col, date_format

appointment_summary_df =appointment_summary_base_df.withColumn(
    "day_name", date_format(col("appointment_date"), "EEEE")
)
appointment_summary_df.display()

appointment_id,appointment_timestamp,appointment_date,appointment_year,appointment_month,appointment_day,patient_id,doctor_id,ingestion_date,patient_name,patient_email,doctor_name,specialization,doctor_contact,procedure_name,procedure_id,total_amount,total_billing,day_name
531,2023-12-23T14:33:46.412Z,2022-06-13,2022,06,13,103,291,2025-08-25T10:14:23.193Z,Lusa Alisia,lusa.alisia@yopmail.com,Gusella,pulmonologists,.@yopmail.com,Electroconvulsive Therapy (ect),615,1144958.0,1144958.0,Monday
455,2023-12-23T14:33:46.41Z,2023-03-15,2023,03,15,103,388,2025-08-25T10:14:23.193Z,Lusa Alisia,lusa.alisia@yopmail.com,Danika,ophthalmology,.@yopmail.com,Insulin Pump Management For Diabetes,904,1144958.0,1144958.0,Wednesday
656,2023-12-23T14:33:46.414Z,2023-07-25,2023,07,25,112,612,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Krystle,pediatrics,.@yopmail.com,null,null,975444.0,975444.0,Tuesday
619,2023-12-23T14:33:46.412Z,2022-06-16,2022,06,16,112,926,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Sashenka,radiology,.@yopmail.com,null,null,975444.0,975444.0,Thursday
443,2023-12-23T14:33:46.408Z,2022-11-27,2022,11,27,112,210,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Dorene,nephrology,.@yopmail.com,null,null,975444.0,975444.0,Sunday
943,2023-12-23T14:33:46.407Z,2020-01-13,2020,01,13,116,889,2025-08-25T10:14:23.193Z,Babita Felecia,babita.felecia@yopmail.com,Paola,infectious disease,.@yopmail.com,null,null,1747344.0,1747344.0,Monday
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,28,121,155,2025-08-25T10:14:23.193Z,Correy Pozzy,correy.pozzy@yopmail.com,Alleen,pulmonologists,.@yopmail.com,Anesthetic Care During Childbirth,804,114657.0,114657.0,Tuesday
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,28,121,155,2025-08-25T10:14:23.193Z,Correy Pozzy,correy.pozzy@yopmail.com,Alleen,pulmonologists,.@yopmail.com,Retinal Surgery,665,114657.0,114657.0,Tuesday
833,2023-12-23T14:33:46.407Z,2023-05-29,2023,05,29,122,800,2025-08-25T10:14:23.193Z,Janenna Croix,janenna.croix@yopmail.com,Hermione,radiology,.@yopmail.com,Sedation For Minor Procedures,959,468251.0,468251.0,Monday
346,2023-12-23T14:33:46.416Z,2021-08-27,2021,08,27,126,844,2025-08-25T10:14:23.193Z,Silvana Zina,silvana.zina@yopmail.com,Tybie,obstetric anesthesiologists,.@yopmail.com,Plastic Surgery,963,null,null,Friday


## First Appointment Flag

In [0]:
from pyspark.sql import Window 
from pyspark.sql.functions import row_number, when,col
# Transformation 3: Flag is_first_appointment for each patient
# Create window specification by patient_id ordered by appointment timestamp
patient_window = Window.partitionBy("patient_id").orderBy("appointment_timestamp")

appointment_summary_df = appointment_summary_df.withColumn(
    "appointment_rank", 
    row_number().over(patient_window)
).withColumn(
    "is_first_appointment", 
    when(col("appointment_rank") == 1, True).otherwise(False)
).drop("appointment_rank")
appointment_summary_df.display()

appointment_id,appointment_timestamp,appointment_date,appointment_year,appointment_month,appointment_day,patient_id,doctor_id,ingestion_date,patient_name,patient_email,doctor_name,specialization,doctor_contact,procedure_name,procedure_id,total_amount,total_billing,day_name,is_first_appointment
455,2023-12-23T14:33:46.41Z,2023-03-15,2023,03,15,103,388,2025-08-25T10:14:23.193Z,Lusa Alisia,lusa.alisia@yopmail.com,Danika,ophthalmology,.@yopmail.com,Insulin Pump Management For Diabetes,904,1144958.0,1144958.0,Wednesday,true
531,2023-12-23T14:33:46.412Z,2022-06-13,2022,06,13,103,291,2025-08-25T10:14:23.193Z,Lusa Alisia,lusa.alisia@yopmail.com,Gusella,pulmonologists,.@yopmail.com,Electroconvulsive Therapy (ect),615,1144958.0,1144958.0,Monday,false
443,2023-12-23T14:33:46.408Z,2022-11-27,2022,11,27,112,210,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Dorene,nephrology,.@yopmail.com,null,null,975444.0,975444.0,Sunday,true
619,2023-12-23T14:33:46.412Z,2022-06-16,2022,06,16,112,926,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Sashenka,radiology,.@yopmail.com,null,null,975444.0,975444.0,Thursday,false
656,2023-12-23T14:33:46.414Z,2023-07-25,2023,07,25,112,612,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Krystle,pediatrics,.@yopmail.com,null,null,975444.0,975444.0,Tuesday,false
943,2023-12-23T14:33:46.407Z,2020-01-13,2020,01,13,116,889,2025-08-25T10:14:23.193Z,Babita Felecia,babita.felecia@yopmail.com,Paola,infectious disease,.@yopmail.com,null,null,1747344.0,1747344.0,Monday,true
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,28,121,155,2025-08-25T10:14:23.193Z,Correy Pozzy,correy.pozzy@yopmail.com,Alleen,pulmonologists,.@yopmail.com,Anesthetic Care During Childbirth,804,114657.0,114657.0,Tuesday,true
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,28,121,155,2025-08-25T10:14:23.193Z,Correy Pozzy,correy.pozzy@yopmail.com,Alleen,pulmonologists,.@yopmail.com,Retinal Surgery,665,114657.0,114657.0,Tuesday,false
833,2023-12-23T14:33:46.407Z,2023-05-29,2023,05,29,122,800,2025-08-25T10:14:23.193Z,Janenna Croix,janenna.croix@yopmail.com,Hermione,radiology,.@yopmail.com,Sedation For Minor Procedures,959,468251.0,468251.0,Monday,true
346,2023-12-23T14:33:46.416Z,2021-08-27,2021,08,27,126,844,2025-08-25T10:14:23.193Z,Silvana Zina,silvana.zina@yopmail.com,Tybie,obstetric anesthesiologists,.@yopmail.com,Plastic Surgery,963,null,null,Friday,true


## Day Bucket (Early / Mid / Late Month)

In [0]:
from pyspark.sql.functions import dayofmonth

appointment_summary_df = appointment_summary_df.withColumn(
    "appointment_day_num", dayofmonth("appointment_date")
).withColumn(
    "day_bucket",
    when(col("appointment_day_num") <= 10, "Early Month")
    .when((col("appointment_day_num") > 10) & (col("appointment_day_num") <= 20), "Mid Month")
    .otherwise("Late Month")
)
appointment_summary_df.display()

appointment_id,appointment_timestamp,appointment_date,appointment_year,appointment_month,appointment_day,patient_id,doctor_id,ingestion_date,patient_name,patient_email,doctor_name,specialization,doctor_contact,procedure_name,procedure_id,total_amount,total_billing,day_name,is_first_appointment,appointment_day_num,day_bucket
455,2023-12-23T14:33:46.41Z,2023-03-15,2023,03,15,103,388,2025-08-25T10:14:23.193Z,Lusa Alisia,lusa.alisia@yopmail.com,Danika,ophthalmology,.@yopmail.com,Insulin Pump Management For Diabetes,904,1144958.0,1144958.0,Wednesday,true,15,Mid Month
531,2023-12-23T14:33:46.412Z,2022-06-13,2022,06,13,103,291,2025-08-25T10:14:23.193Z,Lusa Alisia,lusa.alisia@yopmail.com,Gusella,pulmonologists,.@yopmail.com,Electroconvulsive Therapy (ect),615,1144958.0,1144958.0,Monday,false,13,Mid Month
443,2023-12-23T14:33:46.408Z,2022-11-27,2022,11,27,112,210,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Dorene,nephrology,.@yopmail.com,null,null,975444.0,975444.0,Sunday,true,27,Late Month
619,2023-12-23T14:33:46.412Z,2022-06-16,2022,06,16,112,926,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Sashenka,radiology,.@yopmail.com,null,null,975444.0,975444.0,Thursday,false,16,Mid Month
656,2023-12-23T14:33:46.414Z,2023-07-25,2023,07,25,112,612,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Krystle,pediatrics,.@yopmail.com,null,null,975444.0,975444.0,Tuesday,false,25,Late Month
943,2023-12-23T14:33:46.407Z,2020-01-13,2020,01,13,116,889,2025-08-25T10:14:23.193Z,Babita Felecia,babita.felecia@yopmail.com,Paola,infectious disease,.@yopmail.com,null,null,1747344.0,1747344.0,Monday,true,13,Mid Month
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,28,121,155,2025-08-25T10:14:23.193Z,Correy Pozzy,correy.pozzy@yopmail.com,Alleen,pulmonologists,.@yopmail.com,Anesthetic Care During Childbirth,804,114657.0,114657.0,Tuesday,true,28,Late Month
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,28,121,155,2025-08-25T10:14:23.193Z,Correy Pozzy,correy.pozzy@yopmail.com,Alleen,pulmonologists,.@yopmail.com,Retinal Surgery,665,114657.0,114657.0,Tuesday,false,28,Late Month
833,2023-12-23T14:33:46.407Z,2023-05-29,2023,05,29,122,800,2025-08-25T10:14:23.193Z,Janenna Croix,janenna.croix@yopmail.com,Hermione,radiology,.@yopmail.com,Sedation For Minor Procedures,959,468251.0,468251.0,Monday,true,29,Late Month
346,2023-12-23T14:33:46.416Z,2021-08-27,2021,08,27,126,844,2025-08-25T10:14:23.193Z,Silvana Zina,silvana.zina@yopmail.com,Tybie,obstetric anesthesiologists,.@yopmail.com,Plastic Surgery,963,null,null,Friday,true,27,Late Month


## Day Type (Weekday vs Weekend)

In [0]:
from pyspark.sql.functions import dayofweek

appointment_summary_df = appointment_summary_df.withColumn(
    "day_of_week_num", dayofweek("appointment_date")
).withColumn(
    "day_type",
    when((col("day_of_week_num") == 1) | (col("day_of_week_num") == 7), "Weekend")
    .otherwise("Weekday")
)
appointment_summary_df.display()

appointment_id,appointment_timestamp,appointment_date,appointment_year,appointment_month,appointment_day,patient_id,doctor_id,ingestion_date,patient_name,patient_email,doctor_name,specialization,doctor_contact,procedure_name,procedure_id,total_amount,total_billing,day_name,is_first_appointment,appointment_day_num,day_bucket,day_of_week_num,day_type
455,2023-12-23T14:33:46.41Z,2023-03-15,2023,03,15,103,388,2025-08-25T10:14:23.193Z,Lusa Alisia,lusa.alisia@yopmail.com,Danika,ophthalmology,.@yopmail.com,Insulin Pump Management For Diabetes,904,1144958.0,1144958.0,Wednesday,true,15,Mid Month,4,Weekday
531,2023-12-23T14:33:46.412Z,2022-06-13,2022,06,13,103,291,2025-08-25T10:14:23.193Z,Lusa Alisia,lusa.alisia@yopmail.com,Gusella,pulmonologists,.@yopmail.com,Electroconvulsive Therapy (ect),615,1144958.0,1144958.0,Monday,false,13,Mid Month,2,Weekday
443,2023-12-23T14:33:46.408Z,2022-11-27,2022,11,27,112,210,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Dorene,nephrology,.@yopmail.com,null,null,975444.0,975444.0,Sunday,true,27,Late Month,1,Weekend
619,2023-12-23T14:33:46.412Z,2022-06-16,2022,06,16,112,926,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Sashenka,radiology,.@yopmail.com,null,null,975444.0,975444.0,Thursday,false,16,Mid Month,5,Weekday
656,2023-12-23T14:33:46.414Z,2023-07-25,2023,07,25,112,612,2025-08-25T10:14:23.193Z,Elbertina Dosia,elbertina.dosia@yopmail.com,Krystle,pediatrics,.@yopmail.com,null,null,975444.0,975444.0,Tuesday,false,25,Late Month,3,Weekday
943,2023-12-23T14:33:46.407Z,2020-01-13,2020,01,13,116,889,2025-08-25T10:14:23.193Z,Babita Felecia,babita.felecia@yopmail.com,Paola,infectious disease,.@yopmail.com,null,null,1747344.0,1747344.0,Monday,true,13,Mid Month,2,Weekday
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,28,121,155,2025-08-25T10:14:23.193Z,Correy Pozzy,correy.pozzy@yopmail.com,Alleen,pulmonologists,.@yopmail.com,Anesthetic Care During Childbirth,804,114657.0,114657.0,Tuesday,true,28,Late Month,3,Weekday
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,28,121,155,2025-08-25T10:14:23.193Z,Correy Pozzy,correy.pozzy@yopmail.com,Alleen,pulmonologists,.@yopmail.com,Retinal Surgery,665,114657.0,114657.0,Tuesday,false,28,Late Month,3,Weekday
833,2023-12-23T14:33:46.407Z,2023-05-29,2023,05,29,122,800,2025-08-25T10:14:23.193Z,Janenna Croix,janenna.croix@yopmail.com,Hermione,radiology,.@yopmail.com,Sedation For Minor Procedures,959,468251.0,468251.0,Monday,true,29,Late Month,2,Weekday
346,2023-12-23T14:33:46.416Z,2021-08-27,2021,08,27,126,844,2025-08-25T10:14:23.193Z,Silvana Zina,silvana.zina@yopmail.com,Tybie,obstetric anesthesiologists,.@yopmail.com,Plastic Surgery,963,null,null,Friday,true,27,Late Month,6,Weekday


In [0]:
# Build the final curated DataFrame
final_summary_df = appointment_summary_df.select(
    "appointment_id",
    "appointment_timestamp",
    "appointment_date",
    "appointment_year",
    "appointment_month",
    "patient_id",
    "patient_name",
    "patient_email",
    "doctor_id",
    "doctor_name",
    "specialization",
    "doctor_contact",
    "procedure_id",
    "procedure_name",
    "total_amount",            
    "day_name",                   
    "appointment_day_num",        
    "day_bucket",                 
    "day_of_week_num",            
    "day_type",                   
    "is_first_appointment",      
    "ingestion_date"             
)

# Preview
final_summary_df.display()

appointment_id,appointment_timestamp,appointment_date,appointment_year,appointment_month,patient_id,patient_name,patient_email,doctor_id,doctor_name,specialization,doctor_contact,procedure_id,procedure_name,total_amount,day_name,appointment_day_num,day_bucket,day_of_week_num,day_type,is_first_appointment,ingestion_date
455,2023-12-23T14:33:46.41Z,2023-03-15,2023,03,103,Lusa Alisia,lusa.alisia@yopmail.com,388,Danika,ophthalmology,.@yopmail.com,904,Insulin Pump Management For Diabetes,1144958.0,Wednesday,15,Mid Month,4,Weekday,true,2025-08-25T10:14:23.193Z
531,2023-12-23T14:33:46.412Z,2022-06-13,2022,06,103,Lusa Alisia,lusa.alisia@yopmail.com,291,Gusella,pulmonologists,.@yopmail.com,615,Electroconvulsive Therapy (ect),1144958.0,Monday,13,Mid Month,2,Weekday,false,2025-08-25T10:14:23.193Z
443,2023-12-23T14:33:46.408Z,2022-11-27,2022,11,112,Elbertina Dosia,elbertina.dosia@yopmail.com,210,Dorene,nephrology,.@yopmail.com,null,null,975444.0,Sunday,27,Late Month,1,Weekend,true,2025-08-25T10:14:23.193Z
619,2023-12-23T14:33:46.412Z,2022-06-16,2022,06,112,Elbertina Dosia,elbertina.dosia@yopmail.com,926,Sashenka,radiology,.@yopmail.com,null,null,975444.0,Thursday,16,Mid Month,5,Weekday,false,2025-08-25T10:14:23.193Z
656,2023-12-23T14:33:46.414Z,2023-07-25,2023,07,112,Elbertina Dosia,elbertina.dosia@yopmail.com,612,Krystle,pediatrics,.@yopmail.com,null,null,975444.0,Tuesday,25,Late Month,3,Weekday,false,2025-08-25T10:14:23.193Z
943,2023-12-23T14:33:46.407Z,2020-01-13,2020,01,116,Babita Felecia,babita.felecia@yopmail.com,889,Paola,infectious disease,.@yopmail.com,null,null,1747344.0,Monday,13,Mid Month,2,Weekday,true,2025-08-25T10:14:23.193Z
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,121,Correy Pozzy,correy.pozzy@yopmail.com,155,Alleen,pulmonologists,.@yopmail.com,804,Anesthetic Care During Childbirth,114657.0,Tuesday,28,Late Month,3,Weekday,true,2025-08-25T10:14:23.193Z
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,121,Correy Pozzy,correy.pozzy@yopmail.com,155,Alleen,pulmonologists,.@yopmail.com,665,Retinal Surgery,114657.0,Tuesday,28,Late Month,3,Weekday,false,2025-08-25T10:14:23.193Z
833,2023-12-23T14:33:46.407Z,2023-05-29,2023,05,122,Janenna Croix,janenna.croix@yopmail.com,800,Hermione,radiology,.@yopmail.com,959,Sedation For Minor Procedures,468251.0,Monday,29,Late Month,2,Weekday,true,2025-08-25T10:14:23.193Z
346,2023-12-23T14:33:46.416Z,2021-08-27,2021,08,126,Silvana Zina,silvana.zina@yopmail.com,844,Tybie,obstetric anesthesiologists,.@yopmail.com,963,Plastic Surgery,null,Friday,27,Late Month,6,Weekday,true,2025-08-25T10:14:23.193Z


In [0]:
%sql
create database if not exists healthcare_analysis;

In [0]:
final_summary_df.write.mode("overwrite").format("delta").saveAsTable(
    "healthcare_analysis.patient_appointment_summary"
)

In [0]:
final_summary_df.write.mode("overwrite").parquet(
    f"abfss://destination@projectadf1.dfs.core.windows.net/patient_appointment_summary"
)

In [0]:
%sql
select * from healthcare_analysis.patient_appointment_summary

appointment_id,appointment_timestamp,appointment_date,appointment_year,appointment_month,patient_id,patient_name,patient_email,doctor_id,doctor_name,specialization,doctor_contact,procedure_id,procedure_name,total_amount,day_name,appointment_day_num,day_bucket,day_of_week_num,day_type,is_first_appointment,ingestion_date
455,2023-12-23T14:33:46.41Z,2023-03-15,2023,03,103,Lusa Alisia,lusa.alisia@yopmail.com,388,Danika,ophthalmology,.@yopmail.com,904,Insulin Pump Management For Diabetes,1144958.0,Wednesday,15,Mid Month,4,Weekday,true,2025-08-25T10:14:23.193Z
531,2023-12-23T14:33:46.412Z,2022-06-13,2022,06,103,Lusa Alisia,lusa.alisia@yopmail.com,291,Gusella,pulmonologists,.@yopmail.com,615,Electroconvulsive Therapy (ect),1144958.0,Monday,13,Mid Month,2,Weekday,false,2025-08-25T10:14:23.193Z
443,2023-12-23T14:33:46.408Z,2022-11-27,2022,11,112,Elbertina Dosia,elbertina.dosia@yopmail.com,210,Dorene,nephrology,.@yopmail.com,null,null,975444.0,Sunday,27,Late Month,1,Weekend,true,2025-08-25T10:14:23.193Z
619,2023-12-23T14:33:46.412Z,2022-06-16,2022,06,112,Elbertina Dosia,elbertina.dosia@yopmail.com,926,Sashenka,radiology,.@yopmail.com,null,null,975444.0,Thursday,16,Mid Month,5,Weekday,false,2025-08-25T10:14:23.193Z
656,2023-12-23T14:33:46.414Z,2023-07-25,2023,07,112,Elbertina Dosia,elbertina.dosia@yopmail.com,612,Krystle,pediatrics,.@yopmail.com,null,null,975444.0,Tuesday,25,Late Month,3,Weekday,false,2025-08-25T10:14:23.193Z
943,2023-12-23T14:33:46.407Z,2020-01-13,2020,01,116,Babita Felecia,babita.felecia@yopmail.com,889,Paola,infectious disease,.@yopmail.com,null,null,1747344.0,Monday,13,Mid Month,2,Weekday,true,2025-08-25T10:14:23.193Z
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,121,Correy Pozzy,correy.pozzy@yopmail.com,155,Alleen,pulmonologists,.@yopmail.com,804,Anesthetic Care During Childbirth,114657.0,Tuesday,28,Late Month,3,Weekday,true,2025-08-25T10:14:23.193Z
191,2023-12-23T14:33:46.414Z,2020-07-28,2020,07,121,Correy Pozzy,correy.pozzy@yopmail.com,155,Alleen,pulmonologists,.@yopmail.com,665,Retinal Surgery,114657.0,Tuesday,28,Late Month,3,Weekday,false,2025-08-25T10:14:23.193Z
833,2023-12-23T14:33:46.407Z,2023-05-29,2023,05,122,Janenna Croix,janenna.croix@yopmail.com,800,Hermione,radiology,.@yopmail.com,959,Sedation For Minor Procedures,468251.0,Monday,29,Late Month,2,Weekday,true,2025-08-25T10:14:23.193Z
346,2023-12-23T14:33:46.416Z,2021-08-27,2021,08,126,Silvana Zina,silvana.zina@yopmail.com,844,Tybie,obstetric anesthesiologists,.@yopmail.com,963,Plastic Surgery,null,Friday,27,Late Month,6,Weekday,true,2025-08-25T10:14:23.193Z
